# Obtain filtered player id poses from raw poses (no GT) and court corners (singles only)

Raw poses: top-down multi-pose detection results for all people detected, with bbox conf > 0.6, keypoint conf > 0.3

Desired output: Frame, id, x1, y1, x2, y2, 0_x, 0_y... 16_x, 16_y. If pose not visible, fill with zeros. If one player is not visible, fill with 0s for bbox and pose coords

Approach (singles):

1. Filter out poses with left foot outside of court, with some relaxation boundaries. False positive better than false negative.
2. Pick the top 2 remaining with the highest confidence. Nearer to camera is assigned player 1, further assigned player 2.
3. If less than 2 boxes left in a frame, use the previous frame. (indefinitely prev)If still not found, then input with zeros.


For doubles, we only have the gt pose, since we can't automatically differentiate between players of the same team without GT id labels.

In [1]:
import cv2
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import namedtuple

In [2]:
raw_rootdir = 'raw_pose_bbox/'
crt_rootdir = '../../../datasets/'

In [3]:
raw_file = 'raw_pose_bbox/pose_bbox_pro/match1/1_01_00_pose_bbox.csv'
crt_file = "../../../datasets/court_corners_pro.csv" 

df_raw = pd.read_csv(raw_file)
df_crt = pd.read_csv(crt_file)

In [4]:
df_raw.head()

,Frame,x1,y1,x2,y2,bbox_conf,0_x,0_y,1_x,1_y,...,12_x,12_y,13_x,13_y,14_x,14_y,15_x,15_y,16_x,16_y
0,0,0.471874,0.405537,0.510999,0.583816,0.993157,0.490457,0.420974,0.491763,0.416332,...,0.483928,0.495257,0.495027,0.532398,0.483928,0.530077,0.495027,0.569540,0.484581,0.567218
1,0,0.557423,0.578038,0.607258,0.854553,0.991925,0.578796,0.605582,0.579809,0.601981,...,0.587910,0.697394,0.574746,0.749600,0.585885,0.753201,0.576771,0.800006,0.580821,0.805407
2,0,0.342985,0.305662,0.377258,0.403028,0.819770,0.360656,0.326771,0.362083,0.322333,...,0.354238,0.387624,0.365826,0.413931,0.352990,0.413931,0.368679,0.413931,0.352277,0.413931
3,0,0.131730,0.315437,0.164492,0.406402,0.795672,0.146945,0.336342,0.148278,0.333973,...,0.142615,0.390826,0.153441,0.416588,0.142781,0.416588,0.156106,0.416588,0.142781,0.416588
4,1,0.471799,0.406428,0.511849,0.585271,0.995671,0.490842,0.419585,0.492152,0.416092,...,0.484292,0.495267,0.494771,0.532526,0.483637,0.530197,0.493462,0.569785,0.484947,0.566292


In [5]:
df_crt.head()

,Unnamed: 0,path,tl_x,tl_y,tr_x,tr_y,br_x,br_y,bl_x,bl_y,...,width,height,_tl_x,_tl_y,_tr_x,_tr_y,_br_x,_br_y,_bl_x,_bl_y
0,0,../profession_dataset/match1/rally_video/1_02_...,401,380,877,380,1006,668,273,672,...,1280,720,0.313281,0.527778,0.685156,0.527778,0.785937,0.927778,0.213281,0.933333
1,1,../profession_dataset/match1/rally_video/1_02_...,401,380,877,380,1006,668,273,672,...,1280,720,0.313281,0.527778,0.685156,0.527778,0.785937,0.927778,0.213281,0.933333
2,2,../profession_dataset/match1/rally_video/1_06_...,401,380,877,380,1006,668,273,672,...,1280,720,0.313281,0.527778,0.685156,0.527778,0.785937,0.927778,0.213281,0.933333
3,3,../profession_dataset/match1/rally_video/1_02_...,401,380,877,380,1006,668,273,672,...,1280,720,0.313281,0.527778,0.685156,0.527778,0.785937,0.927778,0.213281,0.933333
4,4,../profession_dataset/match1/rally_video/1_03_...,401,380,877,380,1006,668,273,672,...,1280,720,0.313281,0.527778,0.685156,0.527778,0.785937,0.927778,0.213281,0.933333


## Functions

In [6]:
def format_crt_pts(df_crt_rel, relax=0.15):
    # format court points
    crt_dict = df_crt_rel.to_dict('records')
    crt_pts = np.zeros((4,2))
    # fill up full court points
    crt_pts[0][0], crt_pts[0][1], crt_pts[1][0], crt_pts[1][1], crt_pts[2][0], crt_pts[2][1], crt_pts[3][0], crt_pts[3][1] = \
        (1-relax)*crt_dict[0]['_tl_x'], (1-relax)*crt_dict[0]['_tl_y'], \
        (1+relax)*crt_dict[0]['_tr_x'], (1-relax)*crt_dict[0]['_tr_y'], \
        (1+relax)*crt_dict[0]['_br_x'], (1+relax)*crt_dict[0]['_br_y'], \
        (1-relax)*crt_dict[0]['_bl_x'], (1+relax)*crt_dict[0]['_bl_y']
    # fill up court points for only the near half
    crt_pts_half_near = crt_pts.copy()
    crt_pts_half_near[0][0], crt_pts_half_near[0][1] = 0.5*(crt_pts[0][0] + crt_pts[3][0]), 0.5*(crt_pts[0][1] + crt_pts[3][1])
    crt_pts_half_near[1][0], crt_pts_half_near[1][1] = 0.5*(crt_pts[1][0] + crt_pts[2][0]), 0.5*(crt_pts[1][1] + crt_pts[2][1])
    # fill up court points for only the far half
    crt_pts_half_far = crt_pts.copy()
    crt_pts_half_far[3][0], crt_pts_half_far[3][1] = crt_pts_half_near[0][0], crt_pts_half_near[0][1]
    crt_pts_half_far[2][0], crt_pts_half_far[2][1] = crt_pts_half_near[1][0], crt_pts_half_near[1][1]

    crt_pts = np.float32(crt_pts.reshape((-1,1,2)))
    crt_pts_half_near = np.float32(crt_pts_half_near.reshape((-1,1,2)))
    crt_pts_half_far = np.float32(crt_pts_half_far.reshape((-1,1,2)))
    
    return crt_pts, crt_pts_half_near, crt_pts_half_far

In [7]:
def create_dummy_rowdict(fr):
    row = {}
    row['Frame'] = fr
    row['x1'], row['y1'], row['x2'], row['y2'], row['bbox_conf'] = 0, 0, 0, 0, 0
    for p in range(17):
        row[str(p) + '_x'], row[str(p) + '_y'] = 0, 0
        
    return row

In [8]:
def get_filtered_pose(df_raw, df_crt, crt_orig_path, relax=0.15):
    df_crt_rel = df_crt[df_crt['path'] == crt_orig_path][['_tl_x', '_tl_y', '_tr_x', '_tr_y', '_br_x', '_br_y', '_bl_x', '_bl_y']]
    
    # format court points
    crt_pts, crt_pts_half_near, crt_pts_half_far = format_crt_pts(df_crt_rel, relax=relax)

    total_frame = df_raw['Frame'].to_numpy()[-1] + 1
    outrows_list = []
    for fr in range(total_frame):
        df_raw_tmp = df_raw[df_raw['Frame'] == fr].sort_values(by=['bbox_conf'], ascending=False) # sort by confidence
        listdict_raw = df_raw_tmp.to_dict('records')

        # select rows that lie within court
        selrows_list = []
        for rowi in listdict_raw:
            left_foot_coord = (rowi['16_x'], rowi['16_y'])
            # check if point is in crt. 1- in, 0 -on line, -1 out
            in_crt = cv2.pointPolygonTest(crt_pts, left_foot_coord, False)
            if in_crt != -1:
                selrows_list.append(rowi)

        # identify near and far row
        # CASE 1
        if len(selrows_list) >= 2:
            # check which row is which id based on distance from camera
            if selrows_list[0]['16_y'] > selrows_list[1]['16_y']:
                near_row, far_row = selrows_list[0], selrows_list[1]
            else:
                near_row, far_row = selrows_list[1], selrows_list[0]
        # CASE 2
        elif len(selrows_list) == 1:
            left_foot_coord = (selrows_list[0]['16_x'], selrows_list[0]['16_y'])
            # check if point is in crt. 1- in, 0 -on line, -1 out
            in_near_crt = cv2.pointPolygonTest(crt_pts_half_near, left_foot_coord, False)
            if in_near_crt != -1:
                near_row = selrows_list[0]

                # reuse far row from previous frame, or set to zero if not found
                try:
                    far_row['Frame'] = fr
                except NameError:
                    far_row = create_dummy_rowdict(fr)
            else:
                far_row = selrows_list[0]

                # reuse near row from previous frame, or set to zero if not found
                try:
                    near_row['Frame'] = fr
                except NameError:
                    near_row = create_dummy_rowdict(fr)
        # CASE 3
        else:
            # reuse near and far rows from previous
            try:
                near_row['Frame'] = fr
                far_row['Frame'] = fr
            except NameError:
                far_row = create_dummy_rowdict(fr)
                near_row = create_dummy_rowdict(fr)

        near_row['id'] = 1
        far_row['id'] = 2

        outrows_list.append(near_row.copy())
        outrows_list.append(far_row.copy())

    df_out = pd.DataFrame(outrows_list)
    return df_out

In [9]:
def save_filtered_pose(raw_rootdir, crtfile, out_rootdir, relax=0.15):
    if 'pro' in raw_rootdir:
        crt_orig_rootpath = '../profession_dataset'
    elif 'am' in raw_rootdir:
        crt_orig_rootpath = '../vids/difficult_dataset'
    df_crt = pd.read_csv(crtfile)
    for matchdir in sorted(os.listdir(raw_rootdir)):
        outdir = os.path.join(out_rootdir, matchdir)
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        for posecsv in sorted(os.listdir(os.path.join(raw_rootdir, matchdir))):
            basename = posecsv.split('_pose_bbox.csv')[0]
            crt_orig_path = os.path.join(crt_orig_rootpath, matchdir, 'rally_video', basename + '.mp4')
            rawcsv = os.path.join(raw_rootdir, matchdir, posecsv)
            outcsv = os.path.join(outdir, basename + '_filtered2pose.csv')
            print(outcsv, ' saved')

            df_raw = pd.read_csv(rawcsv)

            df_out = get_filtered_pose(df_raw, df_crt, crt_orig_path, relax=relax)
            df_out.to_csv(outcsv, index=False)

## am-singles

In [11]:
raw_rootdir_am_singles = os.path.join(raw_rootdir, 'pose_bbox_am_singles')
out_rootdir_am_singles = 'filtered2_pose_bbox/am_singles_filtered2pose/'
crt_rootdir_am = os.path.join(crt_rootdir, 'court_corners_am.csv')

save_filtered_pose(raw_rootdir_am_singles, crt_rootdir_am, out_rootdir_am_singles, relax=0.15)

filtered2_pose_bbox/am_singles_filtered2pose/match24/1_00_01_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_01_01_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_01_02_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_01_03_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_01_04_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_02_04_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_03_04_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_03_05_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_04_05_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match24/1_05_05_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match25/1_01_00_filtered2pose.csv  saved
filtered2_pose_bbox/am_singles_filtered2pose/match25/1

## pro

In [12]:
raw_rootdir_pro = os.path.join(raw_rootdir, 'pose_bbox_pro')
out_rootdir_pro = 'filtered2_pose_bbox/pro_filtered2pose/'
crt_rootdir_pro = os.path.join(crt_rootdir, 'court_corners_pro.csv')

save_filtered_pose(raw_rootdir_pro, crt_rootdir_pro, out_rootdir_pro, relax=0.15)

filtered2_pose_bbox/pro_filtered2pose/match1/1_01_00_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_02_00_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_02_01_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_02_02_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_02_03_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_02_04_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_03_04_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_03_05_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_03_06_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_06_06_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_06_08_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match1/1_06_09_filtered2pose.csv  saved
filtered2_pose_bbox/pro_filtered2pose/match10/1_03_01_filtered2p